<a href="https://colab.research.google.com/github/IT24102088/Plant-Disease-Detection-System-final-PG-NO-147/blob/main/aimlMobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'ramindusulakkana'
os.environ['KAGGLE_KEY'] = '28949047'
clear_output()

In [ ]:
!kaggle datasets download -d "vipoooool/new-plant-diseases-dataset" -p /content/drive/MyDrive/datasets/

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
100% 2.69G/2.70G [00:17<00:00, 149MB/s]
100% 2.70G/2.70G [00:17<00:00, 165MB/s]


In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/datasets/new-plant-diseases-dataset.zip'
extract_path = '/content/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
import tensorflow as tf
import os
data_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and 'healthy' not in d.lower()]

training_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)

Found 48001 files belonging to 26 classes.


In [ ]:
data_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and 'healthy' not in d.lower()]

validation_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)

Found 12000 files belonging to 26 classes.


In [ ]:
print("Classes used for training:")
print(class_names)

Classes used for training:
['Apple___Cedar_apple_rust', 'Tomato___Late_blight', 'Peach___Bacterial_spot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___Esca_(Black_Measles)', 'Tomato___Target_Spot', 'Pepper,_bell___Bacterial_spot', 'Apple___Black_rot', 'Orange___Haunglongbing_(Citrus_greening)', 'Tomato___Early_blight', 'Corn_(maize)___Northern_Leaf_Blight', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Cherry_(including_sour)___Powdery_mildew', 'Potato___Early_blight', 'Tomato___Tomato_mosaic_virus', 'Tomato___Bacterial_spot', 'Grape___Black_rot', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Leaf_Mold', 'Potato___Late_blight', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Apple___Apple_scab', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch']


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Rescaling
training_set = training_set.map(lambda x, y: (Rescaling(1./255)(x), y))
validation_set = validation_set.map(lambda x, y: (Rescaling(1./255)(x), y))

In [ ]:
from tensorflow.keras import layers
geometric_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(0.1, 0.1),
])

def apply_geometric_augmentation(image, label):
    image = geometric_augmentation(image)
    return image, label

training_set = training_set.map(apply_geometric_augmentation)

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential
from keras.optimizers import AdamW, SGD

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
output = Dense(26, activation='softmax')(x)
model = Model(inputs=base.input, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training_set, validation_data=validation_set, epochs=15)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 905s 591ms/step - accuracy: 0.6625 - loss: 1.1465 - val_accuracy: 0.8776 - val_loss: 0.3934
Epoch 2/15
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 864s 575ms/step - accuracy: 0.8520 - loss: 0.4479 - val_accuracy: 0.8896 - val_loss: 0.3390
Epoch 3/15
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 922s 575ms/step - accuracy: 0.8668 - loss: 0.3984 - val_accuracy: 0.8909 - val_loss: 0.3371
Epoch 4/15
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 851s 567ms/step - accuracy: 0.8762 - loss: 0.3784 - val_accuracy: 0.8941 - val_loss: 0.3230
Epoch 5/15
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 856s 570ms/step - accuracy: 0.8763 - loss: 0.3720 - val_accuracy: 0.8936 - val_loss: 0.3202
Epoch 6/15
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 861s 574ms/step - accuracy: 0.8786 - loss: 0.3650 - val_accuracy: 0.8972 - val_loss: 0.3130
Epoch 7/15
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 857s 571ms/step - accuracy: 0.8779 - loss: 0.3602 - val_accuracy: 0.9037 - val_loss: 0.3063
Epoch 8/15
15

In [ ]:
train_loss,train_acc = model.evaluate(training_set)

1501/1501 ━━━━━━━━━━━━━━━━━━━━ 841s 560ms/step - accuracy: 0.9144 - loss: 0.2570


In [ ]:
val_loss,val_acc = model.evaluate(validation_set)

375/375 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.8988 - loss: 0.3184


In [ ]:
model.save("/content/drive/MyDrive/Models/model26.keras")

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Models/model26.keras")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train/Potato___Late_blight/00695906-210d-4a9d-822e-986a17384115___RS_LB 4026_flipTB.JPG'
img = image.load_img(img_path, target_size=(224, 224))

img_array = image.img_to_array(img)


img_array = np.expand_dims(img_array, axis=0)

preprocessed_image = img_array / 255.0

In [ ]:
prediction = model.predict(preprocessed_image)
prediction,prediction.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step


(array([[4.13948811e-07, 1.15381996e-10, 1.36927292e-06, 4.08359665e-06,
         3.55685215e-09, 1.14371382e-04, 1.31735974e-03, 1.35015580e-04,
         2.94506713e-03, 2.00413517e-08, 1.18181933e-05, 5.01688864e-11,
         1.00095513e-05, 1.19258894e-03, 2.70099150e-07, 4.09165796e-06,
         8.65448868e-09, 1.23074706e-09, 8.39309141e-05, 2.05368522e-09,
         9.93945062e-01, 2.31745551e-04, 1.80933057e-06, 6.44716920e-08,
         2.55173638e-09, 7.79815537e-07]], dtype=float32),
 (1, 26))

In [ ]:
result_index = np.argmax(prediction)
result_index

np.int64(20)

In [ ]:
class_names[result_index]

'Potato___Late_blight'